In [42]:
import pandas as pd
import altair as alt

In [59]:
def call_boardgame_data(cat, mech, pub):
    boardgame_data = pd.read_csv('../data/board_game.csv', parse_dates=['year_published'])
    boardgame_data['year_published'] = pd.to_datetime(boardgame_data['year_published'], format= '%Y')
    if cat != None:
        boardgame_data = boardgame_data[boardgame_data['category']==cat]

    
    return boardgame_data

In [90]:
boardgame_data = pd.read_csv('../data/board_game.csv', parse_dates=['year_published'])
boardgame_data['year_published'] = pd.to_datetime(boardgame_data['year_published'], format= '%Y')

In [12]:
boardgame_data['category'] = boardgame_data['category'].str.split(",")

In [31]:
category_ratings_df = boardgame_data.copy()
category_ratings_df['category'] = category_ratings_df['category'].str.split(",")
category_ratings_df = category_ratings_df.explode('category')

In [32]:
category_ratings_df_cut = category_ratings_df.groupby('category').filter(lambda x: len(x) > 200)

In [33]:

# generate a list of median average rating based on category
category_by_rating = category_ratings_df_cut.groupby('category')["average_rating"].median().sort_values(ascending=False).index.to_list()

In [35]:
# highlighting for Figure 2 category subset
category_ratings_df_cut['highlight'] = False
category_ratings_df_cut.loc[category_ratings_df_cut['category'].isin(['Territory Building', 'Fantasy', 'Adventure', 'Horror', 'Card Game', 'Miniatures', 'World War II', 'Wargame']), 'highlight'] = True

In [37]:
features_df = boardgame_data.copy()
features_df = features_df[(features_df['max_players'] < 20) & (features_df['max_players'] > 0) & (features_df['min_players'] > 0)]
features_df = features_df[(features_df['min_playtime'] <= 1000) & (features_df['max_playtime'] <= 2000)]
features_df.max_players = features_df.max_players.astype(int)
features_df.min_players = features_df.min_players.astype(int)

In [38]:
# re read to get date as non-datetime object
boardgame_data1 = pd.read_csv('../data/board_game.csv')
number_of_categories = boardgame_data1.copy()
number_of_mechanics = boardgame_data1.copy()

# map the number of categories to a column
number_of_categories = number_of_categories.dropna(subset=["category"])
number_of_categories = number_of_categories.dropna(subset=["mechanic"])
number_of_categories["complexity_value"] = [len(x.split(",")) for x in number_of_categories["category"]]
number_of_categories["complexity_type"] = "number of categories"
number_of_categories['year_published'] = pd.to_datetime(number_of_categories['year_published'], format='%Y')

# map the number of mechanics to a column
number_of_mechanics = number_of_mechanics.dropna(subset=["mechanic"])
number_of_mechanics = number_of_mechanics.dropna(subset=["category"])
number_of_mechanics["complexity_value"] = [len(x.split(",")) for x in number_of_mechanics["mechanic"]]
number_of_mechanics["complexity_type"] = "number of mechanics"
number_of_mechanics['year_published'] = pd.to_datetime(number_of_mechanics['year_published'], format='%Y')

In [39]:
# count number of board games produced by each publisher
v = boardgame_data1.publisher.value_counts()

# top 10 most publishers by games produced
top_publishers = set(v[0:10].index)
top_10_publishers_by_count = boardgame_data1[boardgame_data1["publisher"].isin(top_publishers)]

In [75]:
boardgame_data

,game_id,description,image,max_players,max_playtime,min_age,min_players,min_playtime,name,playing_time,...,artist,category,compilation,designer,expansion,family,mechanic,publisher,average_rating,users_rated
0,1,Die Macher is a game about seven sequential po...,//cf.geekdo-images.com/images/pic159509.jpg,5,240,14,3,240,Die Macher,240,...,Marcus Gschwendtner,"Economic,Negotiation,Political",NaN,Karl-Heinz Schmiel,NaN,"Country: Germany,Valley Games Classic Line","Area Control / Area Influence,Auction/Bidding,...","Hans im Glück Verlags-GmbH,Moskito Spiele,Vall...",7.66508,4498
1,2,Dragonmaster is a trick-taking card game based...,//cf.geekdo-images.com/images/pic184174.jpg,4,30,12,3,30,Dragonmaster,30,...,Bob Pepper,"Card Game,Fantasy",NaN,"G. W. ""Jerry"" D'Arcey",NaN,Animals: Dragons,Trick-taking,"E.S. Lowe,Milton Bradley",6.60815,478
2,3,"Part of the Knizia tile-laying trilogy, Samura...",//cf.geekdo-images.com/images/pic3211873.jpg,4,60,10,2,30,Samurai,60,...,Franz Vohwinkel,"Abstract Strategy,Medieval",NaN,Reiner Knizia,NaN,"Asian Theme,Country: Japan,Knizia tile-laying ...","Area Control / Area Influence,Hand Management,...","999 Games,ABACUSSPIELE,Astrel Games,Ceilikan J...",7.44119,12019
3,4,When you see the triangular box and the luxuri...,//cf.geekdo-images.com/images/pic285299.jpg,4,60,12,2,60,Tal der Könige,60,...,NaN,Ancient,NaN,Christian Beierer,NaN,"Country: Egypt,Promotional Board Games","Action Point Allowance System,Area Control / A...",KOSMOS,6.60675,314
4,5,"In Acquire, each player strategically invests ...",//cf.geekdo-images.com/images/pic342163.jpg,6,90,12,3,90,Acquire,90,...,"Scott Okumura,Peter Whitley",Economic,NaN,Sid Sackson,NaN,3M Bookshelf Series,"Hand Management,Stock Holding,Tile Placement","3M,Avalon Hill,Avalon Hill (Hasbro),Dujardin,G...",7.35830,15195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10527,214996,Description from the publisher:&#10;&#10;Silve...,//cf.geekdo-images.com/images/pic3093082.png,2,480,12,1,30,"Silver Bayonet: The First Team in Vietnam, 196...",480,...,Charles Kibler,"Vietnam War,Wargame",NaN,"Gene Billingsley,Mitchell Land",NaN,GMT Operational System,Hex-and-Counter,GMT Games,8.35333,75
10528,215437,"Codex: Card-Time Strategy is a customizable, n...",//cf.geekdo-images.com/images/pic3290122.jpg,5,45,13,2,45,Codex: Card-Time Strategy – Core Set,45,...,NaN,"Card Game,Fantasy,Fighting,Video Game Theme",NaN,David Sirlin,Codex: Card-Time Strategy – Flagstone Dominion...,Fantasy Strike,"Deck / Pool Building,Hand Management,Variable ...",Sirlin Games,8.08780,82
10529,215471,Time to walk about town and take some pictures...,//cf.geekdo-images.com/images/pic3290975.png,4,20,12,2,20,Wind the Film!,20,...,宝井 貴子 (Takako Takarai),Card Game,NaN,Saashi,NaN,NaN,"Hand Management,Set Collection",Saashi & Saashi,7.28016,63
10530,216201,The race is on for the robots of the Robo Rall...,//cf.geekdo-images.com/images/pic3374227.jpg,6,120,12,2,20,Robo Rally (2016),120,...,NaN,"Miniatures,Racing,Science Fiction",NaN,Richard Garfield,NaN,Robots,"Action / Movement Programming,Grid Movement,Mo...","Avalon Hill (Hasbro),Hasbro",7.45871,341


In [106]:
def clean_alt_list(list_):
    if type(list_) is str:
        list_ = list_.replace(",","','")
        list_ = list_.replace("[", "['")
        list_ = list_.replace("]","']")
        return list_
    else:
        pass



In [107]:
boardgame_data['category'] = boardgame_data['category'].apply(clean_alt_list)

In [109]:
boardgame_data['category'][1]

'Card Game","Fantasy'

In [63]:

boardgame_data['Linelabel'] = 'Mean Annual Average User Rating'

def scatter_plot_dates(cat=None, mech=None, pub=None):
    """
    Takes in inputs for x and y axis and creates scatter plot for comparisons

    xcat: string input
    ycat: string input

    returns: altair plot
    """

    pa1 = (
        alt.Chart(call_boardgame_data(cat, mech, pub))
        .mark_circle(size=60, opacity=0.1, color="grey")
        .encode(
            alt.X(
                "year_published", axis=alt.Axis(title=None), scale=alt.Scale(zero=False)
            ),
            alt.Y(
                "average_rating",
                axis=alt.Axis(title='Average Rating',
                    titleFontSize=12,
                    offset=14,
                    titleFontWeight=600,
                ),
            ),
        )
        .properties(
            title=alt.TitleParams(
                text="Figure 1: Game Popularity based on Published Year",
                anchor="start",
                fontSize=20,
                dy=-20,
                dx=20,
            ),
            width=700,
            height=200,
        )
    )
    pa1 = pa1 + (
        alt.Chart(call_boardgame_data(cat, mech, pub))
        .mark_line(color="#1f77b4", size=3)
        .encode(
            x="year_published",
            y="mean(average_rating)",
            opacity=alt.Opacity(
                "Linelabel", legend=alt.Legend(title=None, orient="top-left")
            ),
        )
    )

    return pa1

In [64]:
scatter_plot_dates(cat='Fantasy')

ValueError: Linelabel encoding field is specified without a type; the type cannot be inferred because it does not match any column in the data.